In [2]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
# Install transformers branch for Ministral
!pip install git+https://github.com/huggingface/transformers.git@bf3f0ae70d0e902efab4b8517fce88f6697636ce
!pip install --no-deps trl==0.22.2

In [3]:
%%capture
!pip install trackio

In [25]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from unsloth import FastVisionModel
import torch
max_seq_length = 4096 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastVisionModel.from_pretrained(
    model_name = "unsloth/Ministral-3-3B-Instruct-2512",
    max_seq_length = max_seq_length,
    load_in_4bit = False, # False for LoRA 16bit
    fast_inference = False, # Enable vLLM fast inference
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Unrecognized keys in `rope_parameters` for 'rope_type'='yarn': {'max_position_embeddings'}


==((====))==  Unsloth 2025.12.9: Fast Ministral3 patching. Transformers: 5.0.0.dev0.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


[huggingface_hub.utils._http|WARNING]Warning: You are sending unauthenticated requests to the HF Hub. Please set a HF_TOKEN to enable higher rate limits and faster downloads.
Unrecognized keys in `rope_parameters` for 'rope_type'='yarn': {'max_position_embeddings'}
Unrecognized keys in `rope_parameters` for 'rope_type'='yarn': {'max_position_embeddings'}


Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


...

processor_config.json:   0%|          | 0.00/976 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [5]:
model = FastVisionModel.get_peft_model(
    model,
    r = lora_rank, 
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2, # *2 speeds up training
    use_gradient_checkpointing = "unsloth", # Reduces memory usage
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model.vision_tower.transformer` require gradients


## sudoku implementation

In [6]:
from dataclasses import dataclass, field
from typing import List, Tuple, Optional
import random
import copy

def _is_valid_placement(board: List[List[int]], row: int, col: int, num: int) -> bool:
    """Check if placing num at (row, col) is valid."""
    # Check row
    if num in board[row]:
        return False

    # Check column
    if num in [board[r][col] for r in range(9)]:
        return False

    # Check 3x3 box
    box_row, box_col = 3 * (row // 3), 3 * (col // 3)
    for r in range(box_row, box_row + 3):
        for c in range(box_col, box_col + 3):
            if board[r][c] == num:
                return False

    return True

def _solve_sudoku(board: List[List[int]]) -> bool:
    """Solve sudoku using backtracking (for puzzle generation)."""
    for row in range(9):
        for col in range(9):
            if board[row][col] == 0:
                for num in range(1, 10):
                    if _is_valid_placement(board, row, col, num):
                        board[row][col] = num
                        if _solve_sudoku(board):
                            return True
                        board[row][col] = 0
                return False
    return True

def _generate_complete_board(rng: random.Random) -> List[List[int]]:
    """Generate a complete valid Sudoku board."""
    board = [[0 for _ in range(9)] for _ in range(9)]

    # Fill diagonal 3x3 boxes first (they don't affect each other)
    for box in range(3):
        nums = list(range(1, 10))
        rng.shuffle(nums)
        for i in range(3):
            for j in range(3):
                board[box * 3 + i][box * 3 + j] = nums[i * 3 + j]

    # Solve the rest
    _solve_sudoku(board)
    return board

@dataclass
class SudokuGame:
    difficulty: int = 40  # Number of cells to remove (20=easy, 40=medium, 50=hard)
    seed: Optional[int] = None
    _rng: random.Random = field(init=False, repr=False)
    _board: List[List[int]] = field(init=False, repr=False)
    _solution: List[List[int]] = field(init=False, repr=False)
    _initial_board: List[List[int]] = field(init=False, repr=False)
    _moves: int = field(default=0, init=False, repr=False)
    _state: str = field(default="ongoing", init=False, repr=False)

    def __post_init__(self):
        self._rng = random.Random(self.seed)

        # Generate complete board
        complete_board = _generate_complete_board(self._rng)
        self._solution = copy.deepcopy(complete_board)

        # Remove cells to create puzzle
        self._board = copy.deepcopy(complete_board)
        cells = [(r, c) for r in range(9) for c in range(9)]
        self._rng.shuffle(cells)

        for r, c in cells[:self.difficulty]:
            self._board[r][c] = 0

        self._initial_board = copy.deepcopy(self._board)
        self._update_state()

    def board(self) -> List[List[int]]:
        """Return current board state."""
        return [row[:] for row in self._board]

    def initial_board(self) -> List[List[int]]:
        """Return initial puzzle state."""
        return [row[:] for row in self._initial_board]

    def state(self) -> str:
        """Return game state: 'ongoing', 'success', or 'failed'."""
        return self._state

    def moves(self) -> int:
        """Return number of moves made."""
        return self._moves

    def place_number(self, row: int, col: int, num: int) -> bool:
        """Place a number on the board. Returns True if valid move."""
        # Validate input
        if not (0 <= row < 9 and 0 <= col < 9):
            self._state = "failed"
            return False

        if not (1 <= num <= 9):
            self._state = "failed"
            return False

        # Can't modify initial cells
        if self._initial_board[row][col] != 0:
            self._state = "failed"
            return False
        if self._board[row][col] != 0:
            self._state = "failed"
            return False
        # Check if placement is valid
        if not _is_valid_placement(self._board, row, col, num):
            self._state = "failed"
            return False

        # Place number
        self._board[row][col] = num
        self._moves += 1
        self._update_state()
        return True

    def _update_state(self) -> None:
        """Update game state based on current board."""
        # Check if puzzle is complete
        if all(self._board[r][c] != 0 for r in range(9) for c in range(9)):
            # Verify solution is correct
            if self._board == self._solution:
                self._state = "success"
            else:
                self._state = "failed"
        else:
            self._state = "ongoing"

    def pretty(self, colors: bool = True) -> str:
        """Pretty print the Sudoku board."""
        RESET = "\x1b[0m"
        INITIAL = "\x1b[38;5;45m"   # Cyan for initial numbers
        PLACED = "\x1b[38;5;226m"    # Yellow for placed numbers
        EMPTY = "\x1b[38;5;239m"     # Gray for empty cells

        lines = []
        lines.append("┌───────┬───────┬───────┐")

        for row in range(9):
            row_str = "│ "
            for col in range(9):
                num = self._board[row][col]

                if colors:
                    if num == 0:
                        row_str += f"{EMPTY}.{RESET}"
                    elif self._initial_board[row][col] != 0:
                        row_str += f"{INITIAL}{num}{RESET}"
                    else:
                        row_str += f"{PLACED}{num}{RESET}"
                else:
                    row_str += str(num) if num != 0 else "."

                if col % 3 == 2:
                    row_str += " │ "
                else:
                    row_str += " "

            lines.append(row_str.rstrip())

            if row == 8:
                lines.append("└───────┴───────┴───────┘")
            elif row % 3 == 2:
                lines.append("├───────┼───────┼───────┤")

        return "\n".join(lines)

In [7]:
game = SudokuGame(difficulty=30, seed=42)
print("Initial puzzle:")
print(game.pretty())
print(f"\nState: {game.state()}, Moves: {game.moves()}")

Initial puzzle:
┌───────┬───────┬───────┐
│ 4 . 8 │ . 5 2 │ 6 3 . │
│ . 9 3 │ 4 6 7 │ . . 1 │
│ 6 1 2 │ . 9 8 │ 4 . . │
├───────┼───────┼───────┤
│ 1 . 4 │ . . . │ 7 9 5 │
│ 3 . 9 │ 7 1 . │ 8 2 6 │
│ 7 8 . │ 5 . 9 │ 1 . 3 │
├───────┼───────┼───────┤
│ 2 4 . │ 9 7 . │ . 6 . │
│ 8 3 5 │ 6 4 . │ . 7 . │
│ . . 7 │ 2 . . │ . 1 4 │
└───────┴───────┴───────┘

State: ongoing, Moves: 0


In [8]:
game

SudokuGame(difficulty=30, seed=42)

In [9]:
game.place_number(0, 1, 7)
print("\nAfter placing 7 at (1,0):")
print(game.pretty())
print(f"State: {game.state()}, Moves: {game.moves()}")


After placing 7 at (1,0):
┌───────┬───────┬───────┐
│ 4 7 8 │ . 5 2 │ 6 3 . │
│ . 9 3 │ 4 6 7 │ . . 1 │
│ 6 1 2 │ . 9 8 │ 4 . . │
├───────┼───────┼───────┤
│ 1 . 4 │ . . . │ 7 9 5 │
│ 3 . 9 │ 7 1 . │ 8 2 6 │
│ 7 8 . │ 5 . 9 │ 1 . 3 │
├───────┼───────┼───────┤
│ 2 4 . │ 9 7 . │ . 6 . │
│ 8 3 5 │ 6 4 . │ . 7 . │
│ . . 7 │ 2 . . │ . 1 4 │
└───────┴───────┴───────┘
State: ongoing, Moves: 1


## rl env setup

In [10]:
from typing import Callable
from unsloth import execute_with_time_limit

def _execute_strategy(strategy: Callable, game: SudokuGame):
    """Execute a strategy function on a Sudoku game."""
    assert callable(strategy)

    max_moves = 100
    valid_moves = 0  # Track successful moves

    while game.state() == "ongoing" and valid_moves < max_moves:
        try:
            board = game.board()
            initial = game.initial_board()
            result = strategy(board, initial)

            # Validate result format
            if not isinstance(result, (tuple, list)) or len(result) != 3:
                # Invalid format = immediate fail, but return valid moves made
                return valid_moves, "failed"

            row, col, num = result

            # Validate types
            if not all(isinstance(x, int) for x in [row, col, num]):
                return valid_moves, "failed"

            # Try to place number
            success = game.place_number(row, col, num)

            if success:
                valid_moves += 1  # Count this valid move
            else:
                # Invalid move = game fails, but return valid_moves made so far
                return valid_moves, "failed"

        except Exception:
            return valid_moves, "failed"

    if valid_moves >= max_moves and game.state() == "ongoing":
        return valid_moves, "failed"

    return valid_moves, game.state()

In [11]:
@execute_with_time_limit(10)
def execute_strategy(strategy: Callable, game: SudokuGame):
    """Execute strategy with 10 second time limit."""
    return _execute_strategy(strategy, game)

In [12]:
def simple_strategy(board, initial):
    """Simple strategy: fill first empty cell with 1."""
    for r in range(9):
        for c in range(9):
            if board[r][c] == 0 and initial[r][c] == 0:
                return (r, c, 7)
    return (0, 0, 7)

game = SudokuGame(difficulty=30, seed=42)
try:
    moves, state = execute_strategy(simple_strategy, game)
    print(f"Moves: {moves}, State: {state}")
except TimeoutError as e:
    print(f"Timed out: {e}")

Moves: 1, State: failed


## code execution

In [13]:
from unsloth import check_python_modules, create_locked_down_function

# Test safe code
sample = """
def strategy(board, initial):
    for r in range(9):
        for c in range(9):
            if board[r][c] == 0:
                return (r, c, 1)
    return (0, 0, 1)
"""

ok, info = check_python_modules(sample)
print("Safe Python code?", ok)
print(info)

Safe Python code? True
{'stdlib': [], 'non_stdlib': [], 'relative_imports': 0}


In [14]:
sample = """
def strategy(board, initial):
    import numpy as np
    return (0, 0, 1)
"""

ok, info = check_python_modules(sample)
print("Safe Python code?", ok)
print(info)

Safe Python code? False
{'stdlib': [], 'non_stdlib': ['numpy'], 'relative_imports': 0}


In [15]:
prompt = """
Create a Sudoku solving strategy using only native Python built-in functions without any import statements.
You are given two lists of lists (9x9 grids):
- board: current state (0 means empty)
- initial: starting puzzle (0 means was empty, numbers are fixed)

Return a tuple (row, col, number) for the next move.
- row: 0-8 (row index)
- col: 0-8 (column index)
- number: 1-9 (digit to place)

Only place numbers in cells that are BOTH empty in initial AND empty in board (initial[row][col] == 0 AND board[row][col] == 0)
Use Sudoku rules: no duplicates in rows, columns, or 3x3 boxes.
Output your function in backticks:
```python
def strategy(board, initial):
    # Your logic here
    return (row, col, number)
```
All helper functions must be inside def strategy. Output only the function.
""".strip()

print(prompt)

Create a Sudoku solving strategy using only native Python built-in functions without any import statements.
You are given two lists of lists (9x9 grids):
- board: current state (0 means empty)
- initial: starting puzzle (0 means was empty, numbers are fixed)

Return a tuple (row, col, number) for the next move.
- row: 0-8 (row index)
- col: 0-8 (column index)
- number: 1-9 (digit to place)

Only place numbers in cells that are BOTH empty in initial AND empty in board (initial[row][col] == 0 AND board[row][col] == 0)
Use Sudoku rules: no duplicates in rows, columns, or 3x3 boxes.
Output your function in backticks:
```python
def strategy(board, initial):
    # Your logic here
    return (row, col, number)
```
All helper functions must be inside def strategy. Output only the function.


In [16]:
text = tokenizer.apply_chat_template(
    [{"role": "user", "content": prompt.strip()}],
    tokenize = False,
    add_generation_prompt = True,
)

from transformers import TextStreamer
print("=" * 50)
print("BASE MODEL OUTPUT (before RL training):")
print("=" * 50)
_ = model.generate(
    **tokenizer(images=None,text=text, return_tensors = "pt").to("cuda"),
    temperature = 1.0,
    max_new_tokens = 512,
    streamer = TextStreamer(tokenizer, skip_prompt = False),
)

BASE MODEL OUTPUT (before RL training):
<s><s>[SYSTEM_PROMPT]You are Ministral-3-3B-Instruct-2512, a Large Language Model (LLM) created by Mistral AI, a French startup headquartered in Paris.
You power an AI assistant called Le Chat.
Your knowledge base was last updated on 2023-10-01.
The current date is {today}.

When you're not sure about some information or when the user's request requires up-to-date or specific data, you must use the available tools to fetch the information. Do not hesitate to use tools whenever they can provide a more accurate or complete response. If no relevant tools are available, then clearly state that you don't have the information and avoid making up anything.
If the user's question is not clear, ambiguous, or does not provide enough context for you to accurately answer the question, you do not try to answer it right away and you rather ask the user to clarify their request (e.g. "What are some good restaurants around me?" => "Where are you?" or "When is th

## rewards

In [17]:
def extract_function(text):
    """Extract Python function from markdown code blocks."""
    if text.count("```") >= 2:
        first = text.find("```") + 3
        second = text.find("```", first)
        fx = text[first:second].strip()
        fx = fx.removeprefix("python\n")
        fx = fx[fx.find("def"):]
        if fx.startswith("def strategy(board, initial):"):
            return fx
    return None

In [18]:
def function_works(completions, **kwargs):
    """Reward for generating valid executable Python code."""
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        function = extract_function(response)

        if function is not None:
            ok, info = check_python_modules(function)

        if function is None or "error" in info:
            score = -2.0  # Invalid function
        else:
            try:
                new_strategy = create_locked_down_function(function)
                score = 1.0  # Valid function
            except:
                score = -1.0  # Function has errors

        scores.append(score)
    return scores

In [19]:
def no_cheating(completions, **kwargs):
    """Penalize use of external imports."""
    scores = []
    for completion in completions:
        response = completion[0]["content"]
        function = extract_function(response)

        if function is not None:
            ok, info = check_python_modules(function)
            scores.append(1.0 if ok else -20.0)  # Heavy penalty for cheating
        else:
            scores.append(-1.0)  # Failed to create function

    return scores

In [20]:
import numpy as np

global PRINTER
PRINTER = 0

def strategy_succeeds(completions, **kwargs):
    """Reward valid moves even if strategy eventually fails."""
    global PRINTER
    scores = []

    seed = np.random.randint(10000)
    difficulty = 35
    for completion in completions:
        printed = False
        response = completion[0]["content"]
        function = extract_function(response)

        if PRINTER % 5 == 0:
            printed = True
            print("\n" + "=" * 60)
            print(function)
            print("=" * 60)
        PRINTER += 1

        if function is not None:
            ok, info = check_python_modules(function)

        if function is None or "error" in info:
            scores.append(0)
            continue

        try:
            new_strategy = create_locked_down_function(function)
        except:
            scores.append(0)
            continue

        try:
            game = SudokuGame(difficulty=difficulty, seed=seed)
            valid_moves, game_state = execute_strategy(new_strategy, game)
            if valid_moves == difficulty:
                game_state = "success"

            print(f"\n Valid moves: {valid_moves}, Final state: {game_state}")

            if not printed:
                print("Strategy:")
                print(function[:200] + "..." if len(function) > 200 else function)

            print("\nFinal board:")
            print(game.pretty())

            if game_state == "success":
                scores.append(30.0)  # Solved the puzzle!
            elif valid_moves > 0:
                # Reward based on valid moves made before failure
                # Each valid move is worth 0.2 points
                reward = valid_moves * 0.2
                scores.append(reward)
            else:
                scores.append(-2.0)  # Failed immediately with no valid moves

        except TimeoutError:
            print("Timeout")
            scores.append(-1.0)
        except Exception as e:
            print(f"Exception: {str(e)[:100]}")
            scores.append(-3.0)

    return scores

## dataset

In [21]:
from datasets import Dataset

dataset = Dataset.from_list([
    {
        "prompt": [{"role": "user", "content": prompt.strip()}],
        "answer": 0,
    }
] * 1000)

maximum_length = len(tokenizer.apply_chat_template(
    [{"role": "user", "content": prompt.strip()}],
    add_generation_prompt=True
))

print(f"Maximum prompt length: {maximum_length}")
print("\nDataset sample:")
print(dataset[0])

Maximum prompt length: 3245

Dataset sample:
{'prompt': [{'content': 'Create a Sudoku solving strategy using only native Python built-in functions without any import statements.\nYou are given two lists of lists (9x9 grids):\n- board: current state (0 means empty)\n- initial: starting puzzle (0 means was empty, numbers are fixed)\n\nReturn a tuple (row, col, number) for the next move.\n- row: 0-8 (row index)\n- col: 0-8 (column index)\n- number: 1-9 (digit to place)\n\nOnly place numbers in cells that are BOTH empty in initial AND empty in board (initial[row][col] == 0 AND board[row][col] == 0)\nUse Sudoku rules: no duplicates in rows, columns, or 3x3 boxes.\nOutput your function in backticks:\n```python\ndef strategy(board, initial):\n    # Your logic here\n    return (row, col, number)\n```\nAll helper functions must be inside def strategy. Output only the function.', 'role': 'user'}], 'answer': 0}


In [26]:
max_prompt_length = maximum_length + 1 # + 1 just in case!
max_completion_length = max_seq_length - max_prompt_length

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    temperature = 1.0,
    learning_rate = 5e-5,
    weight_decay = 0.001,
    warmup_steps = 20,
    lr_scheduler_type = "linear",
    optim = "adamw_8bit",
    loss_type="grpo",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 4, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_completion_length,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 200,
    save_steps = 100,
    report_to = "trackio", # TrackIO
    output_dir = "outputs",
)

warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.


Unsloth: We now expect `per_device_train_batch_size` * `gradient_accumulation_steps` * `world_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


In [27]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        function_works,
        no_cheating,
        strategy_succeeds,
    ],
    args = training_args,
    train_dataset = dataset,
)

In [28]:
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1 | Total steps = 200
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 67,502,080 of 3,916,592,128 (1.72% trained)


* Trackio project initialized: huggingface
* Trackio metrics will be synced to Hugging Face Dataset: Swekerr/trackio-dataset
* Creating new space: https://huggingface.co/spaces/Swekerr/trackio
* View dashboard by going to: https://Swekerr-trackio.hf.space/


`generation_config` default values have been modified to match model-specific defaults: {'max_length': 262144}. If this is not desired, please set these values explicitly.


* Created new run: Swekerr-1766911424
Unsloth: Will smartly offload gradients to save VRAM!

def strategy(board, initial):
    def is_valid(row, col, number):
        # Check row
        if any(board[row][i] == number for i in range(9)):
            return False
        # Check column
        if any(board[i][col] == number for i in range(9)):
            return False
        # Check 3x3 box
        box_row, box_col = ((row // 3) * 3, (col // 3) * 3)
        if (
            any(board[r][c] == number for r in range(box_row, box_row + 3))
            for c in range(box_col, box_col + 3)
        ):
            return False
        return True

    def cell_validations(row, col):
        for num in range(1, 10):
            if initial[row][col] != 0 and initial[row][col] != num:
                continue
            if is_valid(row, col, num):
                yield (row, col, num)

    def scan_board(board, initial):
        for row in range(9):
            for col in range(9):
            

Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / function_works / mean,rewards / function_works / std,rewards / no_cheating / mean,rewards / no_cheating / std,rewards / strategy_succeeds / mean,rewards / strategy_succeeds / std
1,0.000000,1.600000,4.298837,522.250000,319.000000,850.000000,0.250000,413.000000,319.000000,558.000000,0.000000,0.250000,1.500000,0.500000,1.000000,0.850000,3.065398
2,-0.000000,-7.000000,10.099504,676.000000,479.000000,850.000000,0.500000,502.000000,479.000000,525.000000,0.000000,-1.250000,1.500000,-5.250000,9.878428,-0.500000,1.000000
3,-0.000000,-0.650000,3.047950,611.750000,337.000000,850.000000,0.500000,373.500000,337.000000,410.000000,0.000000,-0.500000,1.732051,0.000000,1.154701,-0.150000,1.398809
4,0.000001,-4.400000,12.520916,729.750000,494.000000,850.000000,0.500000,609.500000,494.000000,725.000000,0.001096,-0.500000,1.732051,-4.750000,10.210289,0.850000,3.176476
5,0.000102,0.650000,2.773085,654.500000,413.000000,850.000000,0.250000,589.333374,413.000000,848.000000,0.102292,0.250000,1.500000,0.500000,1.000000,-0.100000,1.331666
6,0.000369,-7.750000,9.500000,715.000000,310.000000,850.000000,0.750000,310.000000,310.000000,310.000000,0.369393,-2.000000,0.000000,-5.750000,9.500000,0.000000,0.000000
7,0.001066,2.000000,5.856051,597.250000,276.000000,850.000000,0.500000,344.500000,276.000000,413.000000,1.065600,-0.500000,1.732051,0.000000,1.154701,2.500000,3.048497
8,0.001398,-6.250000,10.594810,554.750000,337.000000,850.000000,0.250000,456.333344,337.000000,531.000000,1.398022,-0.500000,1.732051,-4.750000,10.210289,-1.000000,1.154701
9,0.001438,3.250000,5.619905,484.750000,266.000000,850.000000,0.250000,363.000000,266.000000,433.000000,1.437700,0.250000,1.500000,0.500000,1.000000,2.500000,4.123106
10,0.001327,1.850000,3.700000,554.000000,256.000000,850.000000,0.250000,455.333344,256.000000,755.000000,1.326730,0.500000,1.000000,1.000000,0.000000,0.350000,3.496189



 Valid moves: 0, Final state: failed
Strategy:
def strategy(board, initial):
    def find_empty():
        for row in range(9):
            for col in range(9):
                if initial[row][col] == 0 and board[row][col] == 0:
                  ...

Final board:
┌───────┬───────┬───────┐
│ 7 2 . │ . 1 4 │ 5 6 . │
│ . 3 5 │ 6 2 . │ . . . │
│ 6 . 4 │ . 8 9 │ 3 . 7 │
├───────┼───────┼───────┤
│ . 4 7 │ 1 6 . │ 8 9 . │
│ . . . │ 4 7 . │ 6 1 2 │
│ 1 . 6 │ . 9 5 │ 7 4 . │
├───────┼───────┼───────┤
│ 4 7 2 │ 9 5 . │ 1 3 8 │
│ . . . │ 8 . . │ . 7 . │
│ 8 . . │ . . . │ 2 5 . │
└───────┴───────┴───────┘

None

 Valid moves: 7, Final state: failed
Strategy:
def strategy(board, initial):
    def is_valid(row, col, num):
        # Check row
        if any(board[row][i] == num for i in range(9)):
            return False
        # Check column
        if an...

Final board:
┌───────┬───────┬───────┐
│ 3 1 5 │ 2 8 4 │ 7 9 6 │
│ 7 8 2 │ 1 5 3 │ . . 4 │
│ . 4 6 │ . 7 9 │ 2 1 3 │
├───────┼───────┼───

TrainOutput(global_step=200, training_loss=0.0007240968772384803, metrics={'train_runtime': 17268.7898, 'train_samples_per_second': 0.046, 'train_steps_per_second': 0.012, 'total_flos': 0.0, 'train_loss': 0.0007240968772384803})

In [33]:
text = tokenizer.apply_chat_template(
    [{"role": "user", "content": prompt.strip()}],
    tokenize = False,
    add_generation_prompt = True,
)

from transformers import TextStreamer

_ = model.generate(
    **tokenizer(images=None,text=text, return_tensors = "pt").to("cuda"),
    temperature = 1.0,
    max_new_tokens = 4096,
    streamer = TextStreamer(tokenizer, skip_prompt = False),
)

<s><s>[SYSTEM_PROMPT]You are Ministral-3-3B-Instruct-2512, a Large Language Model (LLM) created by Mistral AI, a French startup headquartered in Paris.
You power an AI assistant called Le Chat.
Your knowledge base was last updated on 2023-10-01.
The current date is {today}.

When you're not sure about some information or when the user's request requires up-to-date or specific data, you must use the available tools to fetch the information. Do not hesitate to use tools whenever they can provide a more accurate or complete response. If no relevant tools are available, then clearly state that you don't have the information and avoid making up anything.
If the user's question is not clear, ambiguous, or does not provide enough context for you to accurately answer the question, you do not try to answer it right away and you rather ask the user to clarify their request (e.g. "What are some good restaurants around me?" => "Where are you?" or "When is the next flight to Tokyo" => "Where do you